In [1]:
import os
import pandas as pd
from ebmdatalab import bq

In [3]:
sql = f"""
WITH
  drugs AS (
  SELECT
    DATE(i.month) AS month, -- Converts TIMESTAMP TO DATE (AS it's always just 1st OF month) 
    regional_team,
    bnf_name,
    chemical,
    SUM(items) AS items,
    SUM(quantity) AS quantity,
  FROM
    ebmdatalab.hscic.normalised_prescribing AS i
  INNER JOIN
    ebmdatalab.hscic.practices prac
  ON
    practice = prac.code
    AND setting = 4
  INNER JOIN
    hscic.bnf AS bnf
  ON
    i.bnf_code = bnf.presentation_code
  WHERE
    i.bnf_code LIKE '0212000AM%' -- Inclisiran
  GROUP BY
    month,
    bnf_name,
    regional_team,
    chemical ),
  list AS (
  SELECT
    DATE(month) AS month,
    regional_team_id,
    SUM(total_list_size) AS total_list_size
  FROM
    hscic.practice_statistics AS stats
  INNER JOIN
    hscic.ccgs AS ccgs
  ON
    stats.pct_id = ccgs.code
  GROUP BY
    month,
    regional_team_id )
SELECT
  drugs.month AS month,
  regional_team,
  chemical,
  bnf_name,
  items,
  quantity,
  total_list_size,
  IEEE_DIVIDE(items, total_list_size) AS rate
FROM
  drugs
INNER JOIN
  list
ON
  drugs.regional_team = list.regional_team_id
  AND drugs.month = list.month 

"""

# Define the CSV path for caching the results.
csv_path = os.path.join('..', 'data', 'inclisiran_list.csv')

# Use the cached_read function from the bq library to run the query.
vtm_matched = bq.cached_read(sql, csv_path=csv_path)

Downloading: 100%|█████████████████████████████████████████████████████████████|
